In [9]:
import logging
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs
import json
from sklearn.model_selection import train_test_split
import torch


In [10]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("MPS is available. Using MPS device.")
else:
    device = torch.device("cpu")
    print("MPS device not found. Using CPU.")

MPS is available. Using MPS device.


In [11]:
context_data_files = [
    "../NLP Processing/after_scraping/Context-Data/fine-tuning-traveltriangle-goa.json",
    "../NLP Processing/after_scraping/Context-Data/fine-tuning-traveltriangle-japan.json",
    "../NLP Processing/after_scraping/Context-Data/fine-tuning-traveltriangle-vietnam.json"
]
dataset_files = [
    "../NLP Processing/after_scraping/four_qns/fine-tuning-dataset-traveltriangle-goa.json",
    "../NLP Processing/after_scraping/four_qns/fine-tuning-dataset-traveltriangle-japan.json",
    "../NLP Processing/after_scraping/four_qns/fine-tuning-dataset-traveltriangle-vietnam.json"
]

context_data = {}
for i, file_path in enumerate(context_data_files):
    with open(file_path, "r") as file:
        context_data[i] = json.load(file)

questions = [
    "What is the name of the attraction?",
    "What is the location of the attraction?",
    "Describe the attraction in detail.",
    "What type of attraction is it? (e.g. historical, natural, amusement, beach)"
]


In [12]:
training_data = []

for i, file_path in enumerate(dataset_files):
    with open(file_path, "r") as file:
        dataset = json.load(file)
        for entry in dataset:
            unique_id = str(i) + str(entry["context_index"])
            ans = {"context": context_data[i][str(entry['context_index'])], "qas": [{"id": unique_id, "is_impossible": True, "question":entry["question"], "answers" : [{"text": entry["answer"], "answer_start":0}]}]}
            training_data.append(ans)

print(len(training_data))
training_data, testing_data = train_test_split(training_data, test_size=0.2, random_state=42)

print(len(training_data))
print(len(testing_data))

536
428
108


In [13]:
model_type = "bert"
model_name = "bert-base-uncased"

model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 8
model_args.evaluate_during_training = True
model_args.n_best_size = 3
model_args.num_train_epochs = 3

In [14]:
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 512,
    "num_train_epochs": 3,
    "evaluate_during_training_steps": 1000,
    "wandb_project": "Question-Answering",
    "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size": 3
}

In [15]:
model = QuestionAnsweringModel(
    model_type,
    model_name,
    args=train_args,
    use_cuda=False
)

/Users/noeltiju/miniforge3/envs/pytorch_finetuning/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
model.train_model(training_data, eval_data=testing_data)

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Training loss,▁
global_step,▁
lr,▁
Training loss,0.00161
global_step,50
lr,3e-05


Epoch 1 of 3:   0%|          | 0/3 [00:10<?, ?it/s]/Users/noeltiju/miniforge3/envs/pytorch_finetuning/lib/python3.10/site-packages/wandb/sdk/wandb_run.py:2310: UserWarning: Run (5ltgvj6y) is finished. The call to `_console_raw_callback` will be ignored. Please make sure that you are using an active run.
  lambda data: self._console_raw_callback("stderr", data),
Epoch 3 of 3: 100%|██████████| 3/3 [30:16<00:00, 605.38s/it]


(180,
 {'global_step': [60, 120, 180],
  'correct': [0, 0, 0],
  'similar': [77, 77, 77],
  'incorrect': [0, 0, 0],
  'train_loss': [0.0017634108662605286,
   0.0010869551915675402,
   0.0010053198784589767],
  'eval_loss': [-6.4500062465667725, -6.747494459152222, -6.806386470794678]})